## Fibonacci

In [1]:
def fib1(n:int) -> int:
    return fib1(n-1) + fib1(n-2)
try:
    fib1(3)
except RecursionError as e:
    print(repr(e))

RecursionError('maximum recursion depth exceeded',)


In [2]:
def fib2(n:int) -> int:
    if n < 2:
        return n
    else:
        return fib2(n-2) + fib2(n-1)
    
fib2(8)

21

In [3]:
from typing import Dict
memo:Dict[int, int] = {0:0, 1:1} #base cases
def fib3(n:int) -> int:
    if n not in memo:
        memo[n] = fib3(n-1) + fib3(n-2)
    return memo[n]
print(fib3(8), fib3(50))

21 12586269025


In [4]:
from functools import lru_cache
@lru_cache(maxsize=None)
def fib4(n: int) -> int:
    if n < 2:
        return n
    else:
        return fib4(n-2) + fib4(n-1)
    
print(fib4(8), fib4(50))

21 12586269025


In [5]:
def fib5(n:int) -> int:
    if n == 0:
        return n
    else:
        last:int = 0
        _next:int = 1
        for _ in range(1,n):
            last, _next = _next, last + _next
        return _next
    
print(fib5(8), fib5(50))

21 12586269025


In [6]:
from typing import Generator
def fib6(n:int) -> int:
    yield 0
    if n> 0: 
        yield 1
    last:int = 0
    _next:int = 1
    for _ in range(1,n):
        last, _next = _next, last + _next
        yield _next
        
list([i for i in fib6(50)])

[0,
 1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181,
 6765,
 10946,
 17711,
 28657,
 46368,
 75025,
 121393,
 196418,
 317811,
 514229,
 832040,
 1346269,
 2178309,
 3524578,
 5702887,
 9227465,
 14930352,
 24157817,
 39088169,
 63245986,
 102334155,
 165580141,
 267914296,
 433494437,
 701408733,
 1134903170,
 1836311903,
 2971215073,
 4807526976,
 7778742049,
 12586269025]

## Trivial Compression

In [36]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)

    def _compress(self, gene: str) -> None:
        self.bit_string: int = 1  # start with sentinel
        for nucleotide in gene.upper():
            self.bit_string <<= 2  # shift left two bits
            if nucleotide == "A":  # change last two bits to 00
                self.bit_string |= 0b00
            elif nucleotide == "C":  # change last two bits to 01
                self.bit_string |= 0b01
            elif nucleotide == "G":  # change last two bits to 10
                self.bit_string |= 0b10
            elif nucleotide == "T":  # change last two bits to 11
                self.bit_string |= 0b11
            else:
                raise ValueError("Invalid Nucleotide:{}".format(nucleotide))

    def decompress(self) -> str:
        gene: str = ""
        for i in range(0, self.bit_string.bit_length() - 1, 2):  # - 1 to exclude sentinel
            bits: int = self.bit_string >> i & 0b11  # get just 2 relevant bits
            if bits == 0b00:  # A
                gene += "A"
            elif bits == 0b01:  # C
                gene += "C"
            elif bits == 0b10:  # G
                gene += "G"
            elif bits == 0b11:  # T
                gene += "T"
            else:
                raise ValueError("Invalid bits:{}".format(bits))
        return gene[::-1]  # [::-1] reverses string by slicing backwards

    def __str__(self) -> str:  # string representation for pretty printing
        return self.decompress()



In [43]:
from sys import getsizeof
original:str = "TAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAG"*100
print("original is {} bytes".format(getsizeof(original)))
compressed:CompressedGene = CompressedGene(original)
print("compressed is {} bytes".format(getsizeof(compressed.bit_string)))
print(compressed)
print("original and decompressed are the same: {}".format(original == compressed.decompress()))

original is 4749 bytes
compressed is 1280 bytes
TAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACCATTAACCGTTATATATATATAGCCATGGACAGTAGTAGGGATTAACC

## Unbreakable Encryption

In [44]:
from secrets import token_bytes
from typing import Tuple

In [53]:
def random_key(length:int) -> int:
    # generate length random bytes
    tb:bytes = token_bytes(length)
    # convert to a bit string in the form of an int
    return int.from_bytes(tb, "big")

In [57]:
def encrypt(original:str) -> Tuple[int, int]:
    original_bytes: bytes = original.encode()
    dummy:int = random_key(len(original_bytes))
    original_key:int = int.from_bytes(original_bytes, "big")
    encrypted: int = original_key^dummy # XOR
    return dummy, encrypted

print(encrypt("fred"))

(2977035838, 3607335770)


In [58]:
def decrypt(key1:int, key2:int) -> str:
    decrypted:int = key1^key2 #XOR
    temp:bytes = decrypted.to_bytes((decrypted.bit_length()+7)//8, "big")
    return temp.decode()

In [59]:
key1, key2 = encrypt("One Time Pad!")
result:str = decrypt(key1, key2)
print(result)

One Time Pad!


## Calcluating Pi

In [60]:
def calculate_pi(n_terms:int) -> float:
    numerator:float = 4.0
    denominator:float = 1.0
    operation:float=1.0
    pi:float = 0.0
    for _ in range(n_terms):
        pi += operation*(numerator/denominator)
        denominator += 2.0
        operation *= -1.0
    return pi

print(calculate_pi(1000000))

3.1415916535897743


## Towers of Hanoi

In [61]:
from typing import TypeVar, Generic, List
T = TypeVar("T")

class Stack(Generic[T]):
    
    def __init__(self) -> None:
        self._container: List[T] = []
            
    def push(self, item:T) -> None:
        self._container.append(item)
        
    def pop(self) -> T:
        return self._container.pop()
    
    def __repr__(self) -> str:
        return repr(self._container)

In [62]:
num_discs:int = 3
tower_a: Stack[int] = Stack()
tower_b: Stack[int] = Stack()
tower_c: Stack[int] = Stack()
for i in range(1, num_discs+1):
    tower_a.push(i)

In [64]:
def hanoi(begin:Stack[int], end:Stack[int], temp:Stack[int], n:int) -> None:
    if n == 1:
        end.push(begin.pop())
    else:
        hanoi(begin, temp, end, n-1)
        hanoi(begin, end, temp, 1)
        hanoi(temp, end, begin, n-1)

In [65]:
hanoi(tower_a, tower_c, tower_b, num_discs)
print(tower_a, tower_b, tower_c)

[] [] [1, 2, 3]


In [68]:
def do_hanoi(num_discs:int) -> list:
    tower_a: Stack[int] = Stack()
    tower_b: Stack[int] = Stack()
    tower_c: Stack[int] = Stack()
    for i in range(1, num_discs+1):
        tower_a.push(i)
    hanoi(tower_a, tower_c, tower_b, num_discs)
    print(tower_a, tower_b, tower_c)

In [69]:
do_hanoi(4)

[] [] [1, 2, 3, 4]


In [70]:
do_hanoi(19)

[] [] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
